In [2]:
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql

from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

#scripts
from src import rank_metrics
from src import helpers
from src import table_encoder
from src import metrics

In [3]:
spark = SparkSession(SparkContext())

In [4]:
data_path = '../data/csv/'
ratings_df = pd.DataFrame(pd.read_csv(os.path.join(data_path, 'ratings.csv')))
movies_df = pd.DataFrame(pd.read_csv(os.path.join(data_path, 'movies.csv')))
tags_df = pd.DataFrame(pd.read_csv(os.path.join(data_path, 'tags.csv')))

In [5]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
ratings_df = ratings_df.drop('timestamp', axis=1)

In [7]:
ratings_df.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


In [8]:
ratings = spark.createDataFrame(ratings_df)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [9]:
training.head()

Row(userId=1, movieId=3, rating=4.0)

In [10]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [11]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("RMSE = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(5)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(5)

RMSE = 1.078084870530962


In [18]:
userRecs_df = helpers.spark_to_pandas(userRecs)


In [19]:
top_movies_and_rankings = helpers.get_top_movies_and_ratings(userRecs_df, 90, movies_df)
top_movie_ratings = list(top_movies_and_rankings.values())
top_movies = list(top_movies_and_rankings.keys())

In [20]:
rank_metrics.ndcg_at_k(top_movie_ratings, 5, 0)

1.0

In [21]:
movies_df[movies_df['movieId']==1111]['title'].values[0]

"Microcosmos (Microcosmos: Le peuple de l'herbe) (1996)"

In [22]:
movies_df[movies_df['movieId']==1111]['title'].index[0]

844

In [23]:
top_movie_ratings

[6.438600540161133,
 6.399845600128174,
 6.153404712677002,
 6.051950931549072,
 6.034645080566406]